# Practica 1 - Landscapes
This dataset represents different countries or inland areas with 14 different classes. Due to their difficulty, the images are real images without any practical pre-treatment, it will be necessary to maximise the characteristics of the images to improve the results.

## Transforming and preparing the dataset

The first problem we find is that images don't have the same dimensions, so we have to scale all the images to the same size in order to train our model. The best size is the minimum of all images, in order to avoid useless pixels of information because if we increase the size of the small images we are causing pixels that didn't exist to appear and alter the results.

In [1]:
%pip install opencv-python

  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl (38.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from PIL import Image
import os

def find_smallest_image_in_directory(directory):
    image_files = [file for file in os.listdir(directory) if file.lower().endswith('.jpg')]

    if not image_files:
        print(f"No images found in the directory {directory}.")
        return None, None

    min_size = float('inf')
    min_image = None

    for filename in image_files:
        image = Image.open(os.path.join(directory, filename))
        width, height = image.size
        current_size = width * height

        if current_size < min_size:
            min_size = current_size
            min_image = filename

    return min_image, min_size

base_directory = "dat/a2/a2/data/train"

# Iterate through subdirectories
for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)

    if os.path.isdir(subdirectory_path):
        min_image, min_size = find_smallest_image_in_directory(subdirectory_path)

        if min_image is not None:
            print(f"Found in {subdirectory}: {min_image}")
            print(f"Dimensions: {Image.open(os.path.join(subdirectory_path, min_image)).size}")
        else:
            print(f"No images found in {subdirectory}.")



As we can see the minimum size is (203, 200). To keep things simple we can escale all the images to a square of size (200,200) 

In [3]:
import os
import cv2
import IPython.display as display
from PIL import Image

def resize_images(type):
    # Directorio base
    base_directory = "dat/a2/a2/data/"+type+"/"

    # Iterar a través de los subdirectorios
    for subdirectory in os.listdir(base_directory):
        subdirectory_path = os.path.join(base_directory, subdirectory)

        if os.path.isdir(subdirectory_path):
            num_total_images = len([file for file in os.listdir(subdirectory_path) if file.lower().endswith('.jpg')])

            for filename in os.listdir(subdirectory_path):
                if filename.lower().endswith('.jpg'):
                    image_path = os.path.join(subdirectory_path, filename)
                    image = cv2.imread(image_path)
                    resized_image = cv2.resize(image, (200, 200)) 
                    cv2.imwrite(image_path, resized_image)

resize_images("train")
resize_images("test")

In [11]:
import os
import cv2

# Función para extraer características HOG de una imagen
def extract_hog_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size = (16, 16)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size, cell_size, cell_size, nbins)
    features = hog.compute(image)
    return features

# Directorio base
base_directory = "dat/a2/a2/data/train/"

# Lista para almacenar características
features_list = []

# Iterar a través de los subdirectorios
for subdirectory in os.listdir(base_directory):
    subdirectory_path = os.path.join(base_directory, subdirectory)

    if os.path.isdir(subdirectory_path):
        num_total_images = len([file for file in os.listdir(subdirectory_path) if file.lower().endswith('.jpg')])

        # Redimensionar y procesar todas las imágenes en cada directorio
        #resize_and_display_images_in_directory(subdirectory_path, num_images=num_total_images)

        # Extraer características de cada imagen
        for filename in os.listdir(subdirectory_path):
            image_path = os.path.join(subdirectory_path, filename)
            features = extract_hog_features(image_path)
            features_list.append(features)
print(features_list)

[array([0.11036614, 0.06572187, 0.04098496, ..., 0.3189859 , 0.2074981 ,
       0.13412058], dtype=float32), array([0.13393098, 0.04609975, 0.05367194, ..., 0.16921146, 0.06918313,
       0.11712786], dtype=float32), array([0.00487653, 0.00887323, 0.03280931, ..., 0.12762193, 0.10159623,
       0.10132211], dtype=float32), array([0.18193963, 0.26214567, 0.26214567, ..., 0.03934944, 0.03470585,
       0.09695455], dtype=float32), array([0.04642007, 0.07447116, 0.14279778, ..., 0.18619788, 0.2918216 ,
       0.22830257], dtype=float32), array([0.07808775, 0.12732728, 0.07039703, ..., 0.28205892, 0.2646624 ,
       0.24320024], dtype=float32), array([0.1830554 , 0.19272357, 0.19797772, ..., 0.30928698, 0.09425774,
       0.04550689], dtype=float32), array([0.16099465, 0.20398872, 0.24578384, ..., 0.08024563, 0.14169279,
       0.19510315], dtype=float32), array([0.15243088, 0.22067556, 0.23944576, ..., 0.19853175, 0.02372849,
       0.00952734], dtype=float32), array([0.01376066, 0.006799

In [22]:
from skimage.feature import daisy, local_binary_pattern

def extract_hog_features(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    win_size = (64, 64)
    cell_size = (8, 8)
    block_size = (16, 16)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size, cell_size, cell_size, nbins)
    features = hog.compute(image)
    return features

def extract_daisy_features(image_path):
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    radius = 15
    rings = 3
    histograms = 8
    orientations = 8

    features = daisy(image_gray, step=8, radius=radius, rings=rings, histograms=histograms, orientations=orientations)
    return features.flatten()

def extract_local_binary_patterns(image_path):
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Configuración de los parámetros de la función local_binary_pattern
    radius = 8
    n_points = 24
    method = 'uniform'

    features = local_binary_pattern(image_gray, P=n_points, R=radius, method=method)
    return features.flatten()

def extract_features(image_path, descriptor):
    if(descriptor == 'HOG'):
        extract_hog_features(image_path)
    elif(descriptor == 'Daisy'):
        extract_daisy_features(image_path)
    elif(descriptor == 'LBP'):
        extract_local_binary_patterns(image_path)
    else:
        raise ValueError("Invalid descriptor. Use 'hog', 'daisy', or 'lbp'.")


ModuleNotFoundError: No module named 'skimage'

In [17]:
from os.path import isfile, join, exists
from os import listdir
import numpy as np

path="dat/a2/a2/data/train/"
category_names = listdir(path) 

for i in range(len(category_names)):
    print (category_names[i],'=',i)

print ('total categories:',len(category_names))

def get_filenames(path):
    onlyfiles = [path+f for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

dataset_filenames=[] ##list to keep path of all files in the database
dataset_labels=[]
##category_names.index('store')  list the numeric representation of the category
##category_names[0] list the text representation of the category id
for category in category_names:
    category_filenames=get_filenames(path+category+"/")##get all the filenames in that category
    category_labels=np.ones(len(category_filenames))*category_names.index(category) ##label the category with its index position
    category_labels = category_labels.astype(int)
    dataset_filenames=dataset_filenames+category_filenames
    dataset_labels=dataset_labels+list(category_labels)

print(dataset_filenames)

bedroom = 0
Coast = 1
Forest = 2
Highway = 3
industrial = 4
Insidecity = 5
kitchen = 6
livingroom = 7
Mountain = 8
Office = 9
OpenCountry = 10
store = 11
Street = 12
Suburb = 13
TallBuilding = 14
total categories: 15
['dat/a2/a2/data/train/bedroom/image_0001.jpg', 'dat/a2/a2/data/train/bedroom/image_0002.jpg', 'dat/a2/a2/data/train/bedroom/image_0005.jpg', 'dat/a2/a2/data/train/bedroom/image_0009.jpg', 'dat/a2/a2/data/train/bedroom/image_0010.jpg', 'dat/a2/a2/data/train/bedroom/image_0012.jpg', 'dat/a2/a2/data/train/bedroom/image_0015.jpg', 'dat/a2/a2/data/train/bedroom/image_0022.jpg', 'dat/a2/a2/data/train/bedroom/image_0023.jpg', 'dat/a2/a2/data/train/bedroom/image_0025.jpg', 'dat/a2/a2/data/train/bedroom/image_0027.jpg', 'dat/a2/a2/data/train/bedroom/image_0028.jpg', 'dat/a2/a2/data/train/bedroom/image_0031.jpg', 'dat/a2/a2/data/train/bedroom/image_0032.jpg', 'dat/a2/a2/data/train/bedroom/image_0036.jpg', 'dat/a2/a2/data/train/bedroom/image_0038.jpg', 'dat/a2/a2/data/train/bedroom/

In [18]:
def show_image_with_features(descriptor, image_path):
    extract_features(image_path, descriptor)

    # Mostrar la imagen original
    cv2.imshow('Original Image', image_path)

    # Mostrar la imagen con las características del descriptor aplicadas
    visualized_image = np.zeros_like(image_path)
    visualized_image[:features.shape[0], :features.shape[1]] = features
    cv2.imshow(f'{descriptor.capitalize()} Features', visualized_image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

show_image_with_features("HOG", dataset_filenames[6])

NameError: name 'extract_features' is not defined